# Introduction/Business Problem

Singapore is an island nation of foodies. Though a small country, it is cosmopolitan and there is a huge variety of food available from different cuisines and cultures.

Due to the Covid-19 pandemic, travellers arriving in Singapore would have to serve a period of quarantine, known as Stay Home Notice (SHN). Non-residents would have to serve their SHN at a SHN Dedicated Facility (SDF) such as a hotel. They are provided with meals but also allowed to order takeaway from food places. However, many food places only deliver nearby and do not provide island-wide delivery.

As travellers are now allowed <b>book a specific SDF Suite</b> (at a higher cost), Foursquare's data would be useful for travellers to decide where to book their stay <b>if they would like a greater variety of food</b>, especially since they have to stay for a minimum of 14 days.

# Data

The data required for this project is:

#### 1) Location data of SDFs

The list of suite options is available from Singapore authorities' <a href="https://safetravel.ica.gov.sg/health/shn/sdfupgrade">webpage</a>. Then, their addresses will be used to find out their coordinates.

For example:
"Swissotel the Stamford" and "Intercontinental Robertson Quay" are two of the hotels.

#### 2) Foursquare data on eateries around SDF Suites

Next, I will look up the venues' Foursquare's data.

For example: I will search for "restaurant" around the vicinity of each hotel.

#### 3) Map data

I will then plot the restaurants around the SDFs.

#### 4) Ratings data (if necessary)

If there are too few restaurants around the SDF Suite, I will retrieve rating data for the restaurants for evaluatation.

# Methodology

Generally, I will retrieve the locations of restaurants around each SDF Suite within 150m radius, and map the restaurants so that travellers are able to see how many restaurants are nearby. 

Assumptions for mapping:
<ul>
    <li>Restaurants will have delivery options available but only for nearby locations (for this exercise I have limited to 150 metres)</li>
    <li>Travellers prefer to order from nearby locations for freshness and to save costs</li>
    <li>SDF Suites serve the same types of travellers (in reality, they cater to travellers from different points of origin.</li>
</ul>

#### 1) Import libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize

import folium

#### 2) Foursquare credentials

In [2]:
CLIENT_ID = '5ZBN0BU1XC5JUTUCSIJS5AJ3SVVMS1DK2QTQIKGL0ERLGDHT'
CLIENT_SECRET = 'NYAACEGT004IN4MQ11LE10Z0T1QHNKGWOEYPD1BWEIOSD0PO'
ACCESS_TOKEN = 'HGVUASMSBC4OY5DBPOWEPYATWRGONRIOGIBRP5CIQSCSVOV1'
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 5ZBN0BU1XC5JUTUCSIJS5AJ3SVVMS1DK2QTQIKGL0ERLGDHT
CLIENT_SECRET:NYAACEGT004IN4MQ11LE10Z0T1QHNKGWOEYPD1BWEIOSD0PO


#### 3) SDF Suites' coordinates and search for restaurants nearby

There are 8 SDF Suites:
<ol>
    <li>Swissotel the Stamford (2 Stamford Road)</li>
    <li>InterContinental Robertson Quay (1 Nanson Road)</li>
    <li>Royal Plaza on Scotts (25 Scotts Road)</li>
    <li>The Regent (1 Cuscaden Road)</li>
    <li>Fairmont Singapore (80 Bras Basah Rd)</li>
    <li>Grand Hyatt (10 Scotts Rd)</li>
    <li>Holiday Inn Singapore Atrium (317 Outram Road)</li>
    <li>Park Avenue Rochester (31 Rochester Drive (North Buona Vista Road))</li>
<ol>

#### 3.1) Swisshotel the Stamford (2 Stamford Road)

In [3]:
#geographical information
swissotel_address = '2 Stamford Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
swissotel_location = geolocator.geocode(swissotel_address)
swissotel_latitude = swissotel_location.latitude
swissotel_longitude = swissotel_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, swissotel_latitude, swissotel_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
swissotel_results = requests.get(url).json()

# assign relevant part of JSON to venues
swissotel_venues = swissotel_results['response']['venues']

# tranform venues into a dataframe
swissotel_dataframe = pd.json_normalize(swissotel_venues)

# keep only columns that include venue name, and anything that is associated with location
swissotel_filtered_columns = ['name', 'categories'] + [col for col in swissotel_dataframe.columns if col.startswith('location.')] + ['id']
swissotel_dataframe_filtered = swissotel_dataframe.loc[:, swissotel_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
swissotel_dataframe_filtered['categories'] = swissotel_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
swissotel_dataframe_filtered.columns = [column.split('.')[-1] for column in swissotel_dataframe_filtered.columns]

swissotel_dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,crossStreet,neighborhood,state,id
0,Bachmann Japanese Restaurant Pte Ltd,Japanese Restaurant,420 North Bridge Rd #02-01,1.293732,103.853943,"[{'label': 'display', 'lat': 1.293732052664295...",76,188727,SG,Singapore,Singapore,"[420 North Bridge Rd #02-01, 188727]",NaN,NaN,NaN,4c493d2db5eec9b63dcd7aa4
1,Nalan Restaurant,Indian Restaurant,B2-54 Capitol Singapore,1.292948,103.851423,"[{'label': 'display', 'lat': 1.292947784069329...",223,178905,SG,Singapore,Singapore,"[B2-54 Capitol Singapore (13 Stamford Road), 1...",13 Stamford Road,NaN,NaN,4d9457521646a35d9e8e33a3
2,Watami Japanese Casual Restaurant,Japanese Restaurant,"#B1-06/07, Raffles City Shopping Centre",1.293658,103.854140,"[{'label': 'display', 'lat': 1.293657709207513...",90,179103,SG,Singapore,Singapore,"[#B1-06/07, Raffles City Shopping Centre (252 ...",252 North Bridge Rd,NaN,NaN,4c93607b6cfea0938079b98b
3,Brotzeit German Bier Bar & Restaurant,German Restaurant,"#01-17, Raffles City Shopping Centre",1.295073,103.853021,"[{'label': 'display', 'lat': 1.295073499260366...",199,179103,SG,Singapore,Singapore,"[#01-17, Raffles City Shopping Centre (252 Nor...",252 North Bridge Rd,NaN,NaN,4b2b1225f964a52038b424e3
4,Shahi Maharani North Indian Restaurant,Indian Restaurant,#03-21B Raffles City Shopping Centre,1.294389,103.853872,"[{'label': 'display', 'lat': 1.294388940238421...",130,179103,SG,Singapore,Singapore,[#03-21B Raffles City Shopping Centre (252 Nor...,252 North Bridge Rd,NaN,NaN,4b058812f964a520deaf22e3


In [4]:
print("There are {} restaurants around Swissotel the Stamford.".format(len(swissotel_dataframe)))

There are 26 restaurants around Swissotel the Stamford.


#### 3.2) InterContinental Robertson Quay (1 Nanson Road)

In [5]:
#geographical location
intercon_address = '1 Nanson Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
intercon_location = geolocator.geocode(intercon_address)
intercon_latitude = intercon_location.latitude
intercon_longitude = intercon_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, intercon_latitude, intercon_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
intercon_results = requests.get(url).json()

# assign relevant part of JSON to venues
intercon_venues = intercon_results['response']['venues']

# tranform venues into a dataframe
intercon_dataframe = pd.json_normalize(intercon_venues)

# keep only columns that include venue name, and anything that is associated with location
intercon_filtered_columns = ['name', 'categories'] + [col for col in intercon_dataframe.columns if col.startswith('location.')] + ['id']
intercon_dataframe_filtered = intercon_dataframe.loc[:, intercon_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
intercon_dataframe_filtered['categories'] = intercon_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
intercon_dataframe_filtered.columns = [column.split('.')[-1] for column in intercon_dataframe_filtered.columns]

intercon_dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,state,id
0,Red House Seafood Restaurant,Seafood Restaurant,#01-14 The Quayside,60 Robertson Quay,1.289698,103.839088,"[{'label': 'display', 'lat': 1.289698336202413...",109,238252,SG,Singapore,Singapore,"[#01-14 The Quayside (60 Robertson Quay), 238252]",NaN,4b51abaef964a520245227e3
1,Ishi Restaurant,Japanese Restaurant,#02-06 Intercontinental Robertson Quay,1 Nanson Road,1.290270,103.838804,"[{'label': 'display', 'lat': 1.290269501929712...",100,238910,SG,River Valley,Singapore,[#02-06 Intercontinental Robertson Quay (1 Nan...,NaN,59ef7173be707820550a178e
2,Tang Lung Restaurant,Dim Sum Restaurant,80 Mohamed Sultan Road,NaN,1.291315,103.840577,"[{'label': 'display', 'lat': 1.291315, 'lng': ...",135,239013,SG,River Valley,Singapore,"[80 Mohamed Sultan Road, 239013]",NaN,5fdc37a34b288e7c47d5038c
3,Long Beach Seafood Restaurant,Seafood Restaurant,60 Robertson Quay,#01-14,1.289107,103.838489,"[{'label': 'display', 'lat': 1.289107, 'lng': ...",202,238252,SG,River Valley,Singapore,"[60 Robertson Quay (#01-14), 238252]",NaN,5f98143b1f67181f2703280b
4,Nakasei sushi restaurant,Japanese Restaurant,"1 Nanson Road 02-02A, Gallery Hotel",NaN,1.290002,103.838713,"[{'label': 'display', 'lat': 1.290002171677681...",120,238909,SG,NaN,Singapore,"[1 Nanson Road 02-02A, Gallery Hotel, 238909]",NaN,4f3119f6e4b013bb06b2d080


In [6]:
print("There are {} restaurants around InterContinental.".format(len(intercon_dataframe)))

There are 10 restaurants around InterContinental.


#### 3.3) Royal Plaza on Scotts (25 Scotts Road)

In [7]:
#geographical information
scotts_address = '25 Scotts Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
scotts_location = geolocator.geocode(scotts_address)
scotts_latitude = scotts_location.latitude
scotts_longitude = scotts_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, scotts_latitude, scotts_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
scotts_results = requests.get(url).json()

# assign relevant part of JSON to venues
scotts_venues = scotts_results['response']['venues']

# tranform venues into a dataframe
scotts_dataframe = pd.json_normalize(scotts_venues)

# keep only columns that include venue name, and anything that is associated with location
scotts_filtered_columns = ['name', 'categories'] + [col for col in scotts_dataframe.columns if col.startswith('location.')] + ['id']
scotts_dataframe_filtered = scotts_dataframe.loc[:, scotts_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
scotts_dataframe_filtered['categories'] = scotts_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
scotts_dataframe_filtered.columns = [column.split('.')[-1] for column in scotts_dataframe_filtered.columns]

scotts_dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,state,id
0,Nanbantei Japanese Restaurant,Japanese Restaurant,"#05-132, Far East Plaza",14 Scotts Rd.,1.307124,103.833250,"[{'label': 'display', 'lat': 1.307124266264200...",106,228217,SG,Singapore,Singapore,"[#05-132, Far East Plaza (14 Scotts Rd.), 228217]",NaN,4c079d410ed3c9286abe797d
1,Max's Restaurant,Fried Chicken Joint,"#01-07, Far East Plaza",14 Scotts Rd,1.307303,103.833396,"[{'label': 'display', 'lat': 1.307302516449818...",129,228213,SG,Singapore,Singapore,"[#01-07, Far East Plaza (14 Scotts Rd), 228213]",NaN,5d88b6f6a3b6ca00085f4b31
2,Nana Thai Restaurant,Thai Restaurant,#04-22 Far East Plaza (14 Scotts Rd),None,1.307260,103.833646,"[{'label': 'display', 'lat': 1.307259509467572...",152,228213,SG,Singapore,Singapore,"[#04-22 Far East Plaza (14 Scotts Rd) (None), ...",NaN,4b5845c2f964a520695028e3
3,The Ship Restaurant & Bar,Steakhouse,"#03-16/17/18, Shaw Centre",1 Scotts Rd,1.306713,103.831266,"[{'label': 'display', 'lat': 1.306713010582182...",121,228208,SG,Singapore,Singapore,"[#03-16/17/18, Shaw Centre (1 Scotts Rd), 228208]",NaN,4b5e7534f964a520918f29e3
4,Wan Hao Chinese Restaurant,Chinese Restaurant,"Level 3, Marriott Singapore Tang Plaza",320 Orchard Rd.,1.305004,103.832916,"[{'label': 'display', 'lat': 1.305004, 'lng': ...",209,238865,SG,Singapore,Singapore,"[Level 3, Marriott Singapore Tang Plaza (320 O...",NaN,4d47e3af847e6dcb176398c4


In [8]:
print("There are {} restaurants around Royal Plaza on Scotts.".format(len(scotts_dataframe)))

There are 25 restaurants around Royal Plaza on Scotts.


#### 3.4) The Regent (1 Cuscaden Road)

In [9]:
#geographical information
regent_address = '1 Cuscaden Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
regent_location = geolocator.geocode(regent_address)
regent_latitude = regent_location.latitude
regent_longitude = regent_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, regent_latitude, regent_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
regent_results = requests.get(url).json()

# assign relevant part of JSON to venues
regent_venues = regent_results['response']['venues']

# tranform venues into a dataframe
regent_dataframe = pd.json_normalize(regent_venues)

# keep only columns that include venue name, and anything that is associated with location
regent_filtered_columns = ['name', 'categories'] + [col for col in regent_dataframe.columns if col.startswith('location.')] + ['id']
regent_dataframe_filtered = regent_dataframe.loc[:, regent_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
regent_dataframe_filtered['categories'] = regent_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
regent_dataframe_filtered.columns = [column.split('.')[-1] for column in regent_dataframe_filtered.columns]

regent_dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,postalCode,crossStreet,city,id
0,Seoul Restaurant (Charcoal barbecue),Korean Restaurant,1 Cuscaden Road,1.304777,103.825300,"[{'label': 'display', 'lat': 1.304776816350086...",53,SG,Singapore,[1 Cuscaden Road],NaN,NaN,NaN,539bda5c498ea6a82328fa17
1,Restaurant :Horizon 11-02,Residential Building (Apartment / Condo),15 Leionie Hill Road Horizon Tower East,1.306480,103.825114,"[{'label': 'display', 'lat': 1.30648, 'lng': 1...",208,SG,Singapore,"[15 Leionie Hill Road Horizon Tower East, 239194]",239194,NaN,NaN,51da9cec498e74faca606bcc
2,Blu Restaurant,Mediterranean Restaurant,"Level 24, Shangri-La Hotel Singapore",1.305543,103.826497,"[{'label': 'display', 'lat': 1.305543086602948...",210,SG,Singapore,"[Level 24, Shangri-La Hotel Singapore (22 Oran...",258350,22 Orange Grove Road,NaN,4c929ac3911b8cfaf8b9eeb5
3,Yantra,Indian Restaurant,"#01-28/33, Tanglin Mall",1.304685,103.824368,"[{'label': 'display', 'lat': 1.304684658694654...",53,SG,Singapore,"[#01-28/33, Tanglin Mall (163 Tanglin Road), 2...",247933,163 Tanglin Road,Singapore,4b68fa58f964a520a4952be3
4,Brasserie Les Saveurs,French Restaurant,The St. Regis Singapore,1.305301,103.826277,"[{'label': 'display', 'lat': 1.305300663789304...",175,SG,Singapore,"[The St. Regis Singapore (29 Tanglin Road), 24...",247911,29 Tanglin Road,Singapore,4b9786a4f964a5206c0735e3


In [10]:
print("There are {} restaurants around The Regent.".format(len(regent_dataframe)))

There are 8 restaurants around The Regent.


#### 3.5) Fairmont Singapore (80 Bras Basah Road)

In [11]:
#geographical information
fairmont_address = '80 Bras Basah Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
fairmont_location = geolocator.geocode(fairmont_address)
fairmont_latitude = fairmont_location.latitude
fairmont_longitude = fairmont_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, fairmont_latitude, fairmont_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
fairmont_results = requests.get(url).json()

# assign relevant part of JSON to venues
fairmont_venues = fairmont_results['response']['venues']

# tranform venues into a dataframe
fairmont_dataframe = pd.json_normalize(fairmont_venues)

# keep only columns that include venue name, and anything that is associated with location
fairmont_filtered_columns = ['name', 'categories'] + [col for col in fairmont_dataframe.columns if col.startswith('location.')] + ['id']
fairmont_dataframe_filtered = fairmont_dataframe.loc[:, fairmont_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
fairmont_dataframe_filtered['categories'] = fairmont_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
fairmont_dataframe_filtered.columns = [column.split('.')[-1] for column in fairmont_dataframe_filtered.columns]

fairmont_dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,crossStreet,state,id
0,Bachmann Japanese Restaurant Pte Ltd,Japanese Restaurant,420 North Bridge Rd #02-01,1.293732,103.853943,"[{'label': 'display', 'lat': 1.293732052664295...",42,188727,SG,Singapore,Singapore,"[420 North Bridge Rd #02-01, 188727]",NaN,NaN,4c493d2db5eec9b63dcd7aa4
1,Asia Grand Restaurant,Chinese Restaurant,331 North Bridge Rd.,1.295826,103.853610,"[{'label': 'display', 'lat': 1.295826107741318...",194,188720,SG,Singapore,Singapore,"[331 North Bridge Rd. (Odean Towers), 188720]",Odean Towers,NaN,4b058813f964a5201db022e3
2,Lei Garden Restaurant,Dim Sum Restaurant,#01-24 Chijmes,1.295109,103.852214,"[{'label': 'display', 'lat': 1.295108998347222...",222,187996,SG,Singapore,Singapore,"[#01-24 Chijmes (30 Victoria Street), 187996]",30 Victoria Street,NaN,4b058813f964a5202db022e3
3,Watami Japanese Casual Restaurant,Japanese Restaurant,"#B1-06/07, Raffles City Shopping Centre",1.293658,103.854140,"[{'label': 'display', 'lat': 1.293657709207513...",55,179103,SG,Singapore,Singapore,"[#B1-06/07, Raffles City Shopping Centre (252 ...",252 North Bridge Rd,NaN,4c93607b6cfea0938079b98b
4,Sin Swee Kee Chicken Rice Restaurant,Asian Restaurant,35 Seah Street,1.295971,103.854893,"[{'label': 'display', 'lat': 1.295971, 'lng': ...",231,188391,SG,Singapore,Singapore,"[35 Seah Street, 188391]",NaN,NaN,4b136376f964a520c39623e3


In [12]:
print("There are {} restaurants around Fairmont Singapore.".format(len(fairmont_dataframe)))

There are 30 restaurants around Fairmont Singapore.


#### 3.6) Grand Hyatt (10 Scotts Rd)

In [13]:
#geographical information
hyatt_address = '10 Scotts Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
hyatt_location = geolocator.geocode(hyatt_address)
hyatt_latitude = hyatt_location.latitude
hyatt_longitude = hyatt_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hyatt_latitude, hyatt_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
hyatt_results = requests.get(url).json()

# assign relevant part of JSON to venues
hyatt_venues = hyatt_results['response']['venues']

# tranform venues into a dataframe
hyatt_dataframe = pd.json_normalize(hyatt_venues)

# keep only columns that include venue name, and anything that is associated with location
hyatt_filtered_columns = ['name', 'categories'] + [col for col in hyatt_dataframe.columns if col.startswith('location.')] + ['id']
hyatt_dataframe_filtered = hyatt_dataframe.loc[:, hyatt_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
hyatt_dataframe_filtered['categories'] = hyatt_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
hyatt_dataframe_filtered.columns = [column.split('.')[-1] for column in hyatt_dataframe_filtered.columns]

hyatt_dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,neighborhood,state,id
0,Nanbantei Japanese Restaurant,Japanese Restaurant,"#05-132, Far East Plaza",14 Scotts Rd.,1.307124,103.833250,"[{'label': 'display', 'lat': 1.307124266264200...",81,228217,SG,Singapore,Singapore,"[#05-132, Far East Plaza (14 Scotts Rd.), 228217]",NaN,NaN,4c079d410ed3c9286abe797d
1,Max's Restaurant,Fried Chicken Joint,"#01-07, Far East Plaza",14 Scotts Rd,1.307303,103.833396,"[{'label': 'display', 'lat': 1.307302516449818...",98,228213,SG,Singapore,Singapore,"[#01-07, Far East Plaza (14 Scotts Rd), 228213]",NaN,NaN,5d88b6f6a3b6ca00085f4b31
2,Nana Thai Restaurant,Thai Restaurant,#04-22 Far East Plaza (14 Scotts Rd),None,1.307260,103.833646,"[{'label': 'display', 'lat': 1.307259509467572...",94,228213,SG,Singapore,Singapore,"[#04-22 Far East Plaza (14 Scotts Rd) (None), ...",NaN,NaN,4b5845c2f964a520695028e3
3,Tatsuya Japanese Restaurant,Japanese Restaurant,Goodwood Park Hotel,22 Scotts Rd.,1.308391,103.833997,"[{'label': 'display', 'lat': 1.308391209623584...",226,238518,SG,Singapore,Singapore,"[Goodwood Park Hotel (22 Scotts Rd.), 238518]",Orchard,NaN,4b6ee15ff964a520c1ce2ce3
4,Wan Hao Chinese Restaurant,Chinese Restaurant,"Level 3, Marriott Singapore Tang Plaza",320 Orchard Rd.,1.305004,103.832916,"[{'label': 'display', 'lat': 1.305004, 'lng': ...",169,238865,SG,Singapore,Singapore,"[Level 3, Marriott Singapore Tang Plaza (320 O...",NaN,NaN,4d47e3af847e6dcb176398c4


In [14]:
print("There are {} restaurants around Grand Hyatt.".format(len(hyatt_dataframe)))

There are 27 restaurants around Grand Hyatt.


#### 3.7) Holiday Inn Singapore Atrium (317 Outram Road)

In [15]:
#geographical information
hinn_address = '317 Outram Road, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
hinn_location = geolocator.geocode(hinn_address)
hinn_latitude = hinn_location.latitude
hinn_longitude = hinn_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hinn_latitude, hinn_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
hinn_results = requests.get(url).json()

# assign relevant part of JSON to venues
hinn_venues = hinn_results['response']['venues']

# tranform venues into a dataframe
hinn_dataframe = pd.json_normalize(hinn_venues)

# keep only columns that include venue name, and anything that is associated with location
hinn_filtered_columns = ['name', 'categories'] + [col for col in hinn_dataframe.columns if col.startswith('location.')] + ['id']
hinn_dataframe_filtered = hinn_dataframe.loc[:, hinn_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
hinn_dataframe_filtered['categories'] = hinn_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
hinn_dataframe_filtered.columns = [column.split('.')[-1] for column in hinn_dataframe_filtered.columns]

hinn_dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,id
0,Grand Shanghai Restaurant 大上海,Chinese Restaurant,"Lv 1, King's Centre",392 Havelock Rd,1.289507,103.835640,"[{'label': 'display', 'lat': 1.289506911814648...",181,169663,SG,Singapore,Singapore,"[Lv 1, King's Centre (392 Havelock Rd), 169663]",4b058814f964a52059b022e3
1,Farasha Prata Paradise Restaurant,Indian Restaurant,92 Zion Rd,NaN,1.289374,103.834088,"[{'label': 'display', 'lat': 1.289374172207325...",33,160092,SG,Singapore,Singapore,"[92 Zion Rd, 160092]",4bdb127663c5c9b6d2302668
2,Rainbow Restaurant,None,317 Outram Road #B2-00,NaN,1.288910,103.833915,"[{'label': 'display', 'lat': 1.28891, 'lng': 1...",24,NaN,SG,Singapore,Singapore,[317 Outram Road #B2-00],4cb2924cc5e6a1cd40b5e6f6
3,Atrium Restaurant,Café,317 Outram Rd,NaN,1.289031,103.834743,"[{'label': 'display', 'lat': 1.289030862934829...",75,169075,SG,Singapore,Singapore,"[317 Outram Rd, 169075]",55948dd1498ed82c3e371692
4,Holiday Inn Singapore Atrium,Hotel,317 Outram Rd.,NaN,1.289008,103.834201,"[{'label': 'display', 'lat': 1.2890076, 'lng':...",16,169075,SG,Singapore,Singapore,"[317 Outram Rd., 169075]",4b05880bf964a520ecad22e3


In [16]:
print("There are {} restaurants around Holiday Inn.".format(len(hinn_dataframe)))

There are 7 restaurants around Holiday Inn.


#### 3.8) Park Avenue Rochester (31 Rochester Drive (North Buona Vista Road))

In [30]:
#geographical information
parka_address = '31 Rochester Drive, Singapore, SG'
geolocator = Nominatim(user_agent="singapore_SDF_explorer")
parka_location = geolocator.geocode(parka_address)
parka_latitude = parka_location.latitude
parka_longitude = parka_location.longitude

#search and get JSON
search_query = 'restaurant'
radius = 200
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, parka_latitude, parka_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
parka_results = requests.get(url).json()

# assign relevant part of JSON to venues
parka_venues = parka_results['response']['venues']

# tranform venues into a dataframe
parka_dataframe = pd.json_normalize(parka_venues)

# keep only columns that include venue name, and anything that is associated with location
parka_filtered_columns = ['name', 'categories'] + [col for col in parka_dataframe.columns if col.startswith('location.')] + ['id']
parka_dataframe_filtered = parka_dataframe.loc[:, parka_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
parka_dataframe_filtered['categories'] = parka_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
parka_dataframe_filtered.columns = [column.split('.')[-1] for column in parka_dataframe_filtered.columns]

parka_dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,neighborhood,state,id
0,Watami Japanese Casual Restaurant,Japanese Restaurant,"#02-16, The Star Vista",1 Vista Exchange Green,1.306938,103.788443,"[{'label': 'display', 'lat': 1.306937830946151...",196,138617,SG,Singapore,Singapore,"[#02-16, The Star Vista (1 Vista Exchange Gree...",NaN,NaN,5034dd351648b78a0ff75b57
1,District 10 Bar & Restaurant,Bar,"#01-42/K3, The Star Vista",1 Vista Exchange Green,1.306365,103.787958,"[{'label': 'display', 'lat': 1.306365030905287...",139,138617,SG,Singapore,Singapore,"[#01-42/K3, The Star Vista (1 Vista Exchange G...",NaN,NaN,5034160ce8891ec5940d472c
2,Segar Restaurant,Chinese Restaurant,The Star Vista,1 Vista Exchange Green,1.306950,103.788341,"[{'label': 'display', 'lat': 1.306949632806041...",197,138617,SG,Singapore,Singapore,"[The Star Vista (1 Vista Exchange Green), 138617]",Central Region,NaN,5822acbbdd43124483ad1ed4
3,Mezza Italian Restaurant,Italian Restaurant,NaN,NaN,1.305177,103.788360,"[{'label': 'display', 'lat': 1.305177, 'lng': ...",2,138638,SG,Commonwealth,Singapore,[138638],NaN,Singapore,593a8e6a123a196a3f7e139c
4,Bachmann Japanese Restaurant,Japanese Restaurant,The Star Vista,NaN,1.306258,103.789470,"[{'label': 'display', 'lat': 1.306258498292261...",174,NaN,SG,NaN,Singapore,[The Star Vista],NaN,NaN,509cc2fde4b044f763413b2c


In [31]:
print("There are {} restaurants around Park Avenue.".format(len(parka_dataframe)))

There are 8 restaurants around Park Avenue.


#### 4) Visualise restaurants

In [19]:
#central singapore coordinates
venues_map = folium.Map(location=[1.2867, 103.8535], zoom_start=13)

#### 4.1) Plot hotel locations

In [34]:
# add a red circle marker to represent the Swissotel
folium.CircleMarker(
    [swissotel_latitude, swissotel_longitude],
    radius=10,
    color='black',
    popup='Swissotel the Stamford - 2 Stamford Road',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the InterContinental
folium.CircleMarker(
    [intercon_latitude, intercon_longitude],
    radius=10,
    color='black',
    popup='InterContinental Robertson Quay -1 Nanson Road',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the Royal Plaza on Scotts
folium.CircleMarker(
    [scotts_latitude, scotts_longitude],
    radius=10,
    color='black',
    popup='Royal Plaza on Scotts - 25 Scotts Road',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the The Regent
folium.CircleMarker(
    [regent_latitude, regent_longitude],
    radius=10,
    color='black',
    popup='The Regent - 1 Cuscaden Road',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the Fairmont Singapore
folium.CircleMarker(
    [fairmont_latitude, fairmont_longitude],
    radius=10,
    color='black',
    popup='Fairmont Singapore - 80 Bras Basah Rd',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the Grand Hyatt
folium.CircleMarker(
    [hyatt_latitude, hyatt_longitude],
    radius=10,
    color='black',
    popup='Grand Hyatt - 10 Scotts Rd',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the Holiday Inn Singapore Atrium
folium.CircleMarker(
    [hinn_latitude, hinn_longitude],
    radius=10,
    color='black',
    popup='Holiday Inn - 317 Outram Road',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add a red circle marker to represent the Park Avenue Rochester
folium.CircleMarker(
    [parka_latitude, parka_longitude],
    radius=10,
    color='black',
    popup='Park Avenue Rochester - 31 Rochester Drive',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

#### 4.2) Plot restaurant locations

In [36]:
# add Swissotel restaurants as blue circle markers
for lat, lng, label in zip(swissotel_dataframe_filtered.lat, swissotel_dataframe_filtered.lng, swissotel_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8
    ).add_to(venues_map)

# add Intercontinenal restaurants as green circle markers
for lat, lng, label in zip(intercon_dataframe_filtered.lat, intercon_dataframe_filtered.lng, intercon_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add Royal Plaza restaurants as red circle markers
for lat, lng, label in zip(scotts_dataframe_filtered.lat, scotts_dataframe_filtered.lng, scotts_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.8
    ).add_to(venues_map)

# add Regent restaurants as purple circle markers
for lat, lng, label in zip(regent_dataframe_filtered.lat, regent_dataframe_filtered.lng, regent_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.8
    ).add_to(venues_map)

# add Fairmont restaurants as blue circle markers
for lat, lng, label in zip(fairmont_dataframe_filtered.lat, fairmont_dataframe_filtered.lng, fairmont_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8
    ).add_to(venues_map)

# add Hyatt restaurants as red circle markers
for lat, lng, label in zip(hyatt_dataframe_filtered.lat, hyatt_dataframe_filtered.lng, hyatt_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.8
    ).add_to(venues_map)

# add Holiday Inn restaurants as darkred circle markers
for lat, lng, label in zip(hinn_dataframe_filtered.lat, hinn_dataframe_filtered.lng, hinn_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='darkred',
        popup=label,
        fill = True,
        fill_color='darkred',
        fill_opacity=0.8
    ).add_to(venues_map)

# add Park Avenue restaurants as orange circle markers
for lat, lng, label in zip(parka_dataframe_filtered.lat, parka_dataframe_filtered.lng, parka_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.8
    ).add_to(venues_map)

In [37]:
venues_map

#### Find out unique restaurants in blue cluster - Fairmont and Swissotel

In [53]:
bluedots = pd.concat([fairmont_dataframe,swissotel_dataframe])
bluedots2 = bluedots['id']
print("There are {} blue dots.".format(bluedots2.nunique()))

There are 36 blue dots.


#### Find out the unique restaurants in red cluster - Royal Plaza on Scotts and Grand Hyatt

In [54]:
reddots = pd.concat([scotts_dataframe,hyatt_dataframe])
reddots2 = reddots['id']
print("There are {} red dots.".format(reddots2.nunique()))

There are 32 red dots.


# Discussion

#### Results

The above steps were taken to mark restaurants within a 150 metre radius of SDF Suites.

The SDF Suites are marked in black and neighbouring restaurants are marked in different colours. There are two SDF suites located very close to each other and their neighbouring restaurants were marked with the same colour. Thus, as can be seen from the Folium map, the eight SDF Suite locations and their neighbouring restaurants form six clusters, marked out in: red, blue, purple, green, dark red and orange.

The orange cluster is far away from the other clusters located in the central area of Singapore, and there are only eight restaurants identified in this cluster. 

The purple, dark red and green clusters are located more centrally. However, there are not many restaurants identified in these three clusters.

<img src="https://raw.githubusercontent.com/malfoyette/Coursera_Capstone/main/sixclusters.png">

The blue and red clusters have far more restaurants, 36 and 32 respectively.

<img src="https://raw.githubusercontent.com/malfoyette/Coursera_Capstone/main/bluecluster.png">

<img src="https://raw.githubusercontent.com/malfoyette/Coursera_Capstone/main/redcluster.png">

# Conclusion

#### Ranking

If SDF Suites are ranked by the number of restaurants nearby:

| Rank  | SDF Suite |
| -------- | -------- |
| 1 | Swissotel the Stamford<br>Fairmont Singapore |
| 2 | Royal Plaza on Scotts<br>Grand Hyatt |
| 3 | InterContinental Robertson Quay<br>The Regent<br>Holiday Inn Singapore Atrium |
| 4 | Park Avenue Rochester |

Further research: the research can be extended to find out which cluster (blue or red) has more amenities or delivery services available other than delivery food.